In [1]:
using Makie, TopOpt, LinearAlgebra, StatsFuns
using TopOpt.TopOptProblems.Visualization: visualize

┌ Info: Precompiling Makie [ee78f7c6-11fb-53f2-987a-cfe4a2b5a57a]
└ @ Base loading.jl:1313
┌ Info: Precompiling TopOpt [53a1e1a5-51bb-58a9-8a02-02056cc81109]
└ @ Base loading.jl:1313


## Parameter setup

In [20]:
E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 4.0 # filter radius
xmin = 0.0001 # minimum density
problem_size = (30, 10, 10)
V = 0.5 # maximum volume fraction
p = 4.0 # penalty

x0 = fill(V, prod(problem_size)); # initial design

## Define a new problem

In [5]:
using Ferrite
using TopOpt
using TopOpt.TopOptProblems: RectilinearGrid, Metadata
using TopOpt.TopOptProblems: left, right, bottom, middley, middlez,
    nnodespercell, nfacespercell, find_black_and_white, find_varind
using TopOpt.Utilities: @params

@params struct NewPointLoadCantilever{dim, T, N, M} <: StiffnessTopOptProblem{dim, T}
    rect_grid::RectilinearGrid{dim, T, N, M}
    E::T
    ν::T
    ch::ConstraintHandler{<:DofHandler{dim, <:Cell{dim,N,M}, T}, T}
    load_dict::Dict{Int, Vector{T}}
    black::AbstractVector
    white::AbstractVector
    varind::AbstractVector{Int}
    metadata::Metadata
end

function NewPointLoadCantilever(::Type{Val{CellType}}, nels::NTuple{dim,Int}, sizes::NTuple{dim}, 
    E = 1.0, ν = 0.3, force = 1.0) where {dim, CellType}
    iseven(nels[2]) && (length(nels) < 3 || iseven(nels[3])) || throw("Grid does not have an even number of elements along the y and/or z axes.")

    _T = promote_type(eltype(sizes), typeof(E), typeof(ν), typeof(force))
    if _T <: Integer
        T = Float64
    else
        T = _T
    end
    if CellType === :Linear || dim === 3
        rect_grid = RectilinearGrid(Val{:Linear}, nels, T.(sizes))
    else
        rect_grid = RectilinearGrid(Val{:Quadratic}, nels, T.(sizes))
    end

    if haskey(rect_grid.grid.facesets, "fixed_all") 
        pop!(rect_grid.grid.facesets, "fixed_all")
    end
    addnodeset!(rect_grid.grid, "fixed_all", x -> left(rect_grid, x));
    
    if haskey(rect_grid.grid.nodesets, "down_force") 
        pop!(rect_grid.grid.nodesets, "down_force")
    end
    if dim == 3
        addnodeset!(rect_grid.grid, "down_force", x -> right(rect_grid, x) && 
            bottom(rect_grid, x));
            #  && middlez(rect_grid, x));
    else
        addnodeset!(rect_grid.grid, "down_force", x -> right(rect_grid, x) && 
            right(rect_grid, x) && middley(rect_grid, x));
    end

    # Create displacement field u
    dh = DofHandler(rect_grid.grid)
    if CellType === :Linear || dim === 3
        push!(dh, :u, dim) # Add a displacement field
    else
        ip = Lagrange{2, RefCube, 2}()
        push!(dh, :u, dim, ip) # Add a displacement field        
    end
    close!(dh)
    
    ch = ConstraintHandler(dh)

    dbc = Dirichlet(:u, getnodeset(rect_grid.grid, "fixed_all"), (x,t) -> zeros(T, dim), collect(1:dim))
    add!(ch, dbc)
    close!(ch)
    t = T(0)
    Ferrite.update!(ch, t)

    metadata = Metadata(dh)
    load_dict = Dict{Int, Vector{T}}()
    for fnode in getnodeset(rect_grid.grid, "down_force")
    	load_dict[fnode] = [0, -force, 0]
    end

    N = nnodespercell(rect_grid)
    M = nfacespercell(rect_grid)

    black, white = find_black_and_white(dh)
    varind = find_varind(black, white)
    
    return NewPointLoadCantilever(rect_grid, E, ν, ch, load_dict, black, white, varind, metadata)
end

# used in FEA to determine default quad order
# we don't assume the problem struct has `rect_grid` to define its grid
TopOptProblems.nnodespercell(p::NewPointLoadCantilever) = nnodespercell(p.rect_grid)

# ! important, used for specification!
function TopOptProblems.getcloaddict(p::NewPointLoadCantilever{dim, T}) where {dim, T}
    # f = T[0, -p.force, 0]
    # fnode = Tuple(getnodeset(p.rect_grid.grid, "down_force"))[1]
    # return Dict{Int, Vector{T}}(fnode => f)
    return p.load_dict
end

LoadError: invalid redefinition of constant NewPointLoadCantilever

In [21]:
problem = NewPointLoadCantilever(Val{:Linear}, problem_size, (1.0, 1.0, 1.0), E, v, f);

## FEA solvers

In [22]:
solver = FEASolver(Displacement, Direct, problem, xmin = xmin);

In [23]:
solver()
u0 = solver.u

11253-element Vector{Float64}:
    0.0
    0.0
    0.0
   -2.2099446376081686
   -0.9999034775224457
   -1.4202725587371576
   -1.5759577627647248
   -0.42196268331009407
   -1.140815179503329
    0.0
    0.0
    0.0
    0.0
    ⋮
   28.54202289729392
 -106.73217916326222
   -0.19481464550894279
   28.60298050880803
 -112.64039125867208
   -0.1259155145335127
   28.61626113360229
 -118.4816362695016
   -0.08202963419333631
   28.614817608618498
 -124.26831756075559
   -0.05206099717716567

In [24]:
# visualize the deformation!
fig = visualize(problem, u0)
Makie.display(fig)

GLMakie.Screen(...)

In [9]:
# cg_assembly_solver = FEASolver(Displacement, CG, Assembly, problem, xmin = xmin)
# cg_matrix_free_solver = FEASolver(Displacement, CG, MatrixFree, problem, xmin = xmin)

## Construct objective and constraints (function definitions)

In [25]:
cheqfilter = DensityFilter(solver, rmin = rmin)
stress = TopOpt.MicroVonMisesStress(solver)
comp = TopOpt.Compliance(problem, solver)

# minimize compliance
function obj(x)
    return comp(cheqfilter(x))
end

# volume bound
function constr(x)
    return sum(cheqfilter(x)) / length(x) - V
end;

## Run optimization

In [27]:
m = Model(obj)
addvar!(m, zeros(length(x0)), ones(length(x0)))
Nonconvex.add_ineq_constraint!(m, constr)

## enable printout
# Nonconvex.show_residuals[] = true

options = MMAOptions(
    maxiter=300, tol = Tolerance(kkt = 1e-3, x=1e-3, f = 1e-3),
)
TopOpt.setpenalty!(solver, p)

@time r = Nonconvex.optimize(
    m, MMA87(dualoptimizer = ConjugateGradient()),
    x0, options = options,
);

(kkt_residual, ipopt_residual, kkttol) = (224.5665607558163, 224.5665607558163, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (4068.8507082578762, 4068.8507082578762, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (1012.4884898914032, 1012.4884898914032, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (472.4905904385448, 472.4905904385448, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (380.62103963772313, 380.62103963772313, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (170.8736490566089, 170.8736490566089, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (82.13063085633439, 82.13063085633439, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (58.81438158896027, 58.81438158896027, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (36.61017225561352, 36.61017225561352, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (32.79226581626342, 32.79226581626342, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (33.45943307425962, 33.45943307425962, 0.001)
(kkt_residual, ipopt_residual, kkttol

(kkt_residual, ipopt_residual, kkttol) = (0.026984649554503193, 0.026984649554503193, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.024455053617295164, 0.024455053617295164, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.022147386527890944, 0.022147386527890944, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.020692149586423625, 0.020692149586423625, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.02061359717938238, 0.02061359717938238, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.020578013334377587, 0.020578013334377587, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.020556860668659027, 0.020556860668659027, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.02055557515334172, 0.02055557515334172, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.020592949699597796, 0.020592949699597796, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.0206258746925716, 0.0206258746925716, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.020654855288785967, 0.020654855288

(kkt_residual, ipopt_residual, kkttol) = (0.00896678697579567, 0.00896678697579567, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008935934416120617, 0.008935934416120617, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008905283803636976, 0.008905283803636976, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008874830831992142, 0.008874830831992142, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008844571834666581, 0.008844571834666581, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008814503603881363, 0.008814503603881363, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008784623284620707, 0.008784623284620707, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008754928321817346, 0.008754928321817346, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008725416374955941, 0.008725416374955941, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.008696085266766573, 0.008696085266766573, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.00866693295864529, 0.0086669

(kkt_residual, ipopt_residual, kkttol) = (0.006202929081847941, 0.006202929081847941, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.006177204121565083, 0.006177204121565083, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.006148532059540912, 0.006148532059540912, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.006122869878268489, 0.006122869878268489, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.006092250003298716, 0.006092250003298716, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.006066721346911663, 0.006066721346911663, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.0060364315013785586, 0.0060364315013785586, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.006011927080193047, 0.006011927080193047, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.005982450629922198, 0.005982450629922198, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.00595717950572805, 0.00595717950572805, 0.001)
(kkt_residual, ipopt_residual, kkttol) = (0.00592986565382958, 0.00592

In [28]:
# show the optimized results!
@show obj(r.minimizer)
@show constr(r.minimizer)
@show maximum(stress(cheqfilter(r.minimizer)))

topology = cheqfilter(r.minimizer);
fig = visualize(problem; topology = topology)
Makie.display(fig)

obj(r.minimizer) = 5672.452630609205
constr(r.minimizer) = -4.863027863732938e-8
maximum(stress(cheqfilter(r.minimizer))) = 15.145781730644202


GLMakie.Screen(...)

## Call FEA solver on a specific topology

In [29]:
x1 = rand(length(solver.vars))

3000-element Vector{Float64}:
 0.6744720333245862
 0.002782894804438163
 0.906420486659322
 0.42136307559778885
 0.3247256140833463
 0.21136878964521766
 0.13033414057848458
 0.8314091103615038
 0.9561806120221832
 0.5012371772169109
 0.7442685071742989
 0.7736567679981494
 0.07703177251599169
 ⋮
 0.5230521950308498
 0.47072710078765323
 0.1994245932403622
 0.19289823750260826
 0.14723507206886643
 0.9314689578746407
 0.4158403277612539
 0.43432176436104886
 0.6676924717985069
 0.23353910302084668
 0.591924553124733
 0.8547611798904593

In [30]:
solver.vars = x1
solver()
u1 = solver.u

11253-element Vector{Float64}:
     0.0
     0.0
     0.0
     2.31169468621784
    -2.2533328289938144
    -0.9814612290880146
   -12.522133529418308
    -2.3739507419411585
    -7.605813384367817
     0.0
     0.0
     0.0
     0.0
     ⋮
   323.865455245924
 -1134.6230024124666
   -53.21755190906014
   324.50719099654896
 -1198.1697666064651
   -54.565238561763714
   324.41081103323944
 -1260.0565047602151
   -56.876957694252155
   324.3649325067805
 -1321.7512372644308
   -58.869249949931984

In [32]:
# visualize the deformation!
fig = visualize(problem, u1; topology=x1, default_exagg_scale=0.001)
Makie.display(fig)

GLMakie.Screen(...)